In [ ]:
import pandas as pd

url_donnees_depistage = "https://www.data.gouv.fr/fr/datasets/r/426bab53-e3f5-4c6a-9d54-dba4442b3dbc"
df_depistage = pd.read_csv(url_donnees_depistage, sep=";")

#on se débarrasse des colonnes inutiles et qu'on doit recalculer
df_depistage = df_depistage.drop(columns=['cl_age90',"Ti","Tp","Td"])
df_depistage = df_depistage.rename(columns = {'pop':'population','P':'nb patients positifs',
'T':'nb patients testes'})

#conversion des valeurs afin de pouvoir faire des calculs
df_depistage["population"] = df_depistage["population"].str.replace(',','.').astype(float)
df_depistage["nb patients positifs"] = df_depistage["nb patients positifs"].str.replace(',','.').astype(float)
df_depistage["nb patients testes"] = df_depistage["nb patients testes"].str.replace(',','.').astype(float)

df_depistage["dep"] = df_depistage['dep'].astype(str)
df_depistage["mois"] = df_depistage["jour"].astype(str).str[5:7]
df_depistage["annee"] = df_depistage["jour"].astype(str).str[:4]

df_depistage["id"] = df_depistage["dep"] + "_" + df_depistage["annee"] + "_" + df_depistage["mois"]


In [ ]:
df_depistage = df_depistage.groupby("id").agg({"dep":"first","mois":"first","annee":"first","population":"mean","nb patients positifs":"sum",
"nb patients testes":"sum"})

#définition de taux en pourcentage
df_depistage['taux de positivite'] = df_depistage['nb patients positifs']*100/df_depistage['nb patients testes']
df_depistage['taux de depistage'] = df_depistage['nb patients testes']*100/df_depistage['population']
df_depistage["taux d'incidence"] = df_depistage['nb patients positifs']*100/df_depistage['population']

df_depistage.head(10)

In [ ]:
df_test = df_depistage.set_index("dep")
df_test.head(10)

In [ ]:
import pandas as pd
url = "https://www.data.gouv.fr/fr/datasets/r/eceb9fb4-3ebc-4da3-828d-f5939712600a"
df_urgences = pd.read_csv(url, sep=";")

df_urgences = df_urgences[['dep','date_de_passage','sursaud_cl_age_corona','nbre_pass_corona', 'nbre_pass_tot','nbre_hospit_corona','nbre_acte_corona','nbre_acte_tot']]
#on remplit les valeurs non renseignées par 0 
df_urgences.fillna(0)
df_urgences["dep"] = df_urgences['dep'].astype(str)
df_urgences["mois"] = df_urgences["date_de_passage"].astype(str).str[5:7]
df_urgences["annee"] = df_urgences["date_de_passage"].astype(str).str[:4]

df_urgences["id"] = df_urgences["dep"] + "_" + df_urgences["annee"] + "_" + df_urgences["mois"] + "_" + df_urgences["sursaud_cl_age_corona"].astype(str)

df_urgences = df_urgences.groupby("id").agg({"dep":"first","mois":"first","annee":"first",
"sursaud_cl_age_corona":"first","nbre_pass_corona":"sum","nbre_pass_tot":"sum",
"nbre_hospit_corona":"sum","nbre_acte_corona":"sum","nbre_acte_tot":"sum"})

df_urgences.head(10)

In [ ]:
import pandas as pd
url_bis = "https://www.data.gouv.fr/fr/datasets/r/6fadff46-9efd-4c53-942a-54aca783c30c"
df_hosp = pd.read_csv(url_bis, sep = ";")

df_hosp = df_hosp.rename(columns = {'incid_hosp':'nb hospitalisations','incid_rea':'nb reanimations',
'incid_dc':'nb deces','incid_rad':'nb retour au domicile'})
df_hosp["dep"] = df_hosp["dep"].astype(str)
df_hosp["mois"] = df_hosp["jour"].astype(str).str[5:7]
df_hosp["annee"] = df_hosp["jour"].astype(str).str[:4]

df_hosp["id"] = df_hosp["dep"] + "_" + df_hosp["annee"] + "_" + df_hosp["mois"] 
df_hosp = df_hosp.groupby("id").agg({"dep":"first","mois":"first","annee":"first","nb hospitalisations":"sum","nb reanimations":"sum",
 "nb deces":"sum", "nb retour au domicile":"sum"})

df_hosp.head(10)


In [ ]:
!pip install openpyxl

In [29]:
import pandas as pd
from fileopener import *

# import des dataframe grâce au fileopener
file = "donnees.txt"
df_list = dataframe_creator(file)
# ordre des df : hospitalisations, urgences, dépistages, etablissements, espérance de vie, niveaux de vie, pauvreté monétaire, vieillissement, population selon age
df_hosp = df_list[0]
df_urgences = df_list[1]
df_depistage = df_list[2]
df_etab = df_list[3]
df_espvie = df_list[4]
df_nvvie = df_list[5]
df_pauv = df_list[6]
df_vieil = df_list[7]
df_pop = df_list[8]
df_deces = df_list[9]


In [30]:
df_hosp = df_hosp.rename(columns = {'incid_hosp':'nb hospitalisations','incid_rea':'nb reanimations',
'incid_dc':'nb deces','incid_rad':'nb retour au domicile'})
df_hosp["dep"] = df_hosp["dep"].astype(str)
df_hosp["mois"] = df_hosp["jour"].astype(str).str[5:7]
df_hosp["annee"] = df_hosp["jour"].astype(str).str[:4]

df_hosp["id"] = df_hosp["dep"] + "_" + df_hosp["annee"] + "_" + df_hosp["mois"] 
df_hosp = df_hosp.groupby("id").agg({"dep":"first","mois":"first","annee":"first","nb hospitalisations":"sum","nb reanimations":"sum",
 "nb deces":"sum", "nb retour au domicile":"sum"})

"""
#on remet les départements comme index de la df
df_hosp = df_hosp.set_index("dep")
"""


'\n#on remet les départements comme index de la df\ndf_hosp = df_hosp.set_index("dep")\n'

In [32]:
#on retirme les territoires
tom_list = ['977','978','986','987','988','984','989']
df_hosp = df_hosp.drop(df_hosp[df_hosp["dep"].isin(tom_list)].index)
display(df_hosp.head())

,dep,mois,annee,nb hospitalisations,nb reanimations,nb deces,nb retour au domicile
id,,,,,,,
01_2020_03,01,03,2020,131,23,11,34
01_2020_04,01,04,2020,415,79,70,242
01_2020_05,01,05,2020,72,6,15,132
01_2020_06,01,06,2020,17,1,8,48
01_2020_07,01,07,2020,7,2,2,27


In [33]:
dep_list = pd.unique(df_hosp["dep"])
column_hosp = df_hosp.columns.to_list()
index_list = []

for dep in dep_list:
    for annee in range(2019,2024):
        for mois in range(1,13):
            id = f'{dep}_{annee}_{mois:02d}'
            index_list.append(id)

df_hosp_fill = pd.DataFrame(0, index=index_list, columns=column_hosp)


In [34]:
display(df_hosp.head())
display(df_hosp_fill.head())

,dep,mois,annee,nb hospitalisations,nb reanimations,nb deces,nb retour au domicile
id,,,,,,,
01_2020_03,01,03,2020,131,23,11,34
01_2020_04,01,04,2020,415,79,70,242
01_2020_05,01,05,2020,72,6,15,132
01_2020_06,01,06,2020,17,1,8,48
01_2020_07,01,07,2020,7,2,2,27


,dep,mois,annee,nb hospitalisations,nb reanimations,nb deces,nb retour au domicile
01_2019_01,0,0,0,0,0,0,0
01_2019_02,0,0,0,0,0,0,0
01_2019_03,0,0,0,0,0,0,0
01_2019_04,0,0,0,0,0,0,0
01_2019_05,0,0,0,0,0,0,0


In [ ]:

df_hosp_test = pd.concat([df_hosp, df_hosp_fill], ignore_index=False)




,dep,mois,annee,nb hospitalisations,nb reanimations,nb deces,nb retour au domicile
01_2020_03,01,03,2020,131,23,11,34
01_2020_04,01,04,2020,415,79,70,242
01_2020_05,01,05,2020,72,6,15,132
01_2020_06,01,06,2020,17,1,8,48
01_2020_07,01,07,2020,7,2,2,27


In [36]:
df_hosp_test = df_hosp_test.reset_index().drop_duplicates(subset="index", keep="first").set_index("index").sort_index()
display(df_hosp_test.head())

,dep,mois,annee,nb hospitalisations,nb reanimations,nb deces,nb retour au domicile
index,,,,,,,
01_2019_01,0,0,0,0,0,0,0
01_2019_02,0,0,0,0,0,0,0
01_2019_03,0,0,0,0,0,0,0
01_2019_04,0,0,0,0,0,0,0
01_2019_05,0,0,0,0,0,0,0


In [37]:
print(df_hosp_test.nunique())
print(len(df_hosp_test))

dep                      102
mois                      13
annee                      5
nb hospitalisations      905
nb reanimations          300
nb deces                 272
nb retour au domicile    810
dtype: int64
6060
